## Create a movie of a high water event from a PTZ (rotating) WebCOOS webcamera alongside a water level timeseries

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

### Add the package directory to pythonpath so it can be imported correctly:

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '../NWLON_WebCOOS_Synchronizer')))

### Import the module and specify your WebCOOS API Token:

In [ ]:
import nwlon_webcoos_synchronizer as synch
token = '' ## Insert your WebCOOS API Token here, as a string ##

### View the available WebCOOS cameras:

In [ ]:
synch.get_cameras(token)

### Select the Charleston Harbor, SC camera and view available imagery products

In [ ]:
camera = 'Charleston Harbor, SC'
synch.get_products(camera,token)

### Choose a product and view the data inventory for that product:

In [ ]:
product = 'one-minute-stills'
synch.get_inventory(camera,product,token)

### This is a PTZ camera. We will use the ViewSeparator utility in DataWebcamSyncher to train and save a machine-learning model that is able to classify images into a view category. Then we can make the movie for only the view that we are interested in.

### First, specify directories for training and validation images:

In [ ]:
direc_train = 'demo_event_ptz/images_ViewSepTraining_Charleston'
direc_test = 'demo_event_ptz/images_ViewSepTesting_Charleston'

### To perform the training, you need to know how many views the camera rotates between, and assign each one an integer number (e.g. 1,2,3...). Once you have this info, you can initialize the ViewSeparator utility:

In [ ]:
vs = synch.ViewSeparator(camera,token,n_views=7)

### Get the training dataset by labelling images. First, pull 250 images from random times within the available data. Then, assign a label to each image based on its view.

#### Note: Images will attempt to be downloaded for 250 random data points. There may not be images available for every data point, so less than 250 images may be downloaded. It is recommended to use n= more images than you think you will need.

In [ ]:
time_start = '202305090000'
time_stop = '202409251800'
vs.get_random_images(time_start,time_stop,n=250,direc=direc_train)
vs.label(direc_train)

### Train and save the ML model:

In [ ]:
decim_fac=10 # This is a factor by which to decimate image resolution, to prevent memory overload issues. #
model = vs.train(direc_train,decim_fac=decim_fac)
vs.save_model(model,decim_fac,'demo_event_ptz','trained_model')

### Test the model visually on some new images:

In [ ]:
vs.get_random_images(time_start,time_stop,n=30,direc=direc_test)
prediction = vs.predict(model,direc_test,decim_fac=10)
vs.inspect_prediction(direc_test,prediction)

### Synchronize the webcamera imagery with water level data and create a movie for only the view of interest by applying the trained model.
### For view_num, you should use the number you assigned to the "home" view - the one showing the walkway out into the water.

In [ ]:
save_dir = 'demo_event_ptz/images_HWEvent' # Input a directory to save images and the movie #
station=8665530
synchro = synch.synch(station=station,                              # The NWLON station ID
                      camera=camera,                                # The WebCOOS camera name #
                      data_product='water_level',                   # The CO-OPS data product to make a timeseries of #
                      camera_product=product,                       # The WebCOOS image product to use for the movie #
                      value='all',                                  # Can be 'all' or a float value, depending on what you want.
                      time_start='202312170800',                    # Start of the movie in local time at the camera location #
                      time_end='202312171600',                      # End of the movie in local time at the camera location
                      interval=1,                                   # Interval of data and imagery, in minutes. #
                      cutoff=None,                                  # Make the movie of only this many data points. Use with value to get what you want.  #
                      sep_model='demo_event_ptz/trained_model.pkl', # The trained view separation model. #
                      token=token,                                  # Your WebCOOS API Token
                      save_dir=save_dir)                            # The directory in which to save the images and movie

mov = synch.make_movie(synchro,camera,station,view_num=4) # USE YOUR CORRECT view_num #